In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import misc
import os
from data_utils import getDataSet, deprocessImage
import math

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, classes_dic, mean = getDataSet('101_ObjectCategories/', 200, 200)

In [ ]:
print ('X_train shape: {}'.format(X_train.shape))
print ('y_train shape: {}'.format(y_train.shape))
print ('X_val shape: {}'.format(X_val.shape))
print ('y_val shape: {}'.format(y_val.shape))
print ('X_test shape: {}'.format(X_test.shape))
print ('y_test shape: {}'.format(y_test.shape))

In [ ]:
i = np.random.randint(0, 7145)
plt.imshow(np.uint8(deprocessImage(X_train[i], mean)))
print (classes_dic[y_train[i]])

In [ ]:
def AlexNetModel(x, num_classes):
    a1 = tf.layers.conv2d(x, filters=96, kernel_size=(11, 11), strides=(4, 4), kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.ones())
    a2 = tf.layers.batch_normalization(a1)
    a3 = tf.nn.relu(a2)
    a44 = tf.layers.dropout(a3)
    a4 = tf.layers.max_pooling2d(a44, pool_size=(3, 3), strides=(2, 2))
    
    a5 = tf.layers.conv2d(a4, filters=256, kernel_size=(5, 5), kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.ones())
    a6 = tf.layers.batch_normalization(a5)
    a7 = tf.nn.relu(a6)
    a88 = tf.layers.dropout(a7)
    a8 = tf.layers.max_pooling2d(a88, pool_size=(3, 3), strides=(2, 2))
    
    a9 = tf.layers.conv2d(a8, filters=384, kernel_size=(3, 3), activation="relu", kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.ones())
    a10 = tf.layers.dropout(a9)
    a11 = tf.layers.conv2d(a10, filters=384, kernel_size=(3, 3), activation="relu", kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.ones())
    a12 = tf.layers.dropout(a11)
    a13 = tf.layers.conv2d(a12, filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer=tf.initializers.random_normal(stddev=0.01), bias_initializer=tf.initializers.ones())
    a14 = tf.layers.dropout(a13)
    
    a15 = tf.layers.max_pooling2d(a14, pool_size=(3, 3), strides=(2, 2))
    
    a16 = tf.layers.flatten(a15)
    
    a17 = tf.layers.dense(a16, units=4096, activation="relu")
    a18 = tf.layers.dropout(a17)
    a19 = tf.layers.dense(a18, units=4096, activation="relu")
    a20 = tf.layers.dropout(a19)
    
    scores = tf.layers.dense(a20, units=num_classes)
    
    return scores

In [ ]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.cast(tf.argmax(predict,1), tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/float(actual_batch_size)))
            iter_cnt += 1
        total_correct = float(correct)/Xd.shape[0]
        total_loss = np.sum(losses, dtype=float)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None])
is_training = tf.placeholder(tf.bool)

num_classes = len(classes_dic)

y_out = AlexNetModel(X, num_classes)

total_loss = tf.losses.softmax_cross_entropy(tf.one_hot(y,num_classes),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

global_step = tf.Variable(0, trainable=False)
#starter_learning_rate = 0.02
#learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
 #                                          500, 0.96, staircase=True)

optimizer = tf.train.MomentumOptimizer(0.01, 0.9)

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)
    
    
saver = tf.train.Saver()

sess = tf.Session()

sess.run(tf.global_variables_initializer())
train_accs = []
val_accs = []
max_val_acc = 0.0
for i in range(50):    
    print('Training')
    loss, acc = run_model(sess,y_out,mean_loss,X_train,y_train,1,128,100,train_step,True)
    train_accs.append(acc)
    print('Validation')
    loss, acc = run_model(sess,y_out,mean_loss,X_val,y_val,1,128)
    val_accs.append(acc)
    if acc > max_val_acc:
        max_val_acc = acc
        saver.save(sess, './model_tensorflow.ckpt')

In [ ]:
print (max_val_accl_acc)

In [ ]:
t = range(50)
plt.plot(t, train_accs, 'r--', t, val_accs, 'b--')
plt.show()

In [ ]:
print('Test')
run_model(sess,y_out,mean_loss,X_test,y_test,1,64)